In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dptos = gpd.read_file('../carto/deptos/departamentos.shp')
grid = gpd.read_file('../carto/grilla/GrillaHexPB.shp')

#dptos = gpd.read_file('../carto/d.shp')
#grid = gpd.read_file('../carto/g.shp')


In [3]:
dptos = dptos.to_crs(epsg=5347)
grid = grid.to_crs(epsg=5347)
dptos['area_depto'] = dptos.geometry.area
grid['area_grid'] = grid.geometry.area
dptos.head()

,link,codpcia,departamen,provincia,color_psid,geometry,area_depto
0,06035,06,Avellaneda,Buenos Aires,2,"POLYGON ((5652734.281733191 6167138.182415018,...",5.727835e+07
1,06805,06,Tigre,Buenos Aires,4,(POLYGON ((5636553.511274694 6203816.275459369...,3.932309e+08
2,06882,06,ZÃ¡rate,Buenos Aires,4,"POLYGON ((5571741.346744739 6258426.326411969,...",1.190238e+09
3,06427,06,La Matanza,Buenos Aires,3,"POLYGON ((5634864.664052583 6164643.855624371,...",3.283928e+08
4,06560,06,Moreno,Buenos Aires,3,"POLYGON ((5607994.058435936 6181191.040708226,...",1.864141e+08


In [ ]:
#f,ax = plt.subplots(figsize=(8,8))
#dptos.plot(ax=ax,linewidth=0.8,edgecolor='red', facecolor='grey',)
#grid.plot(ax=ax,color = 'blue',alpha=0.5)

In [4]:
tabla = gpd.overlay(grid,dptos,how='union').drop_duplicates(subset=['id','area_grid','link','departamen','area_depto'])
tabla['area_union'] = tabla.geometry.area
tabla = tabla.drop('geometry',axis=1)
tabla['peso'] = tabla['area_union'] / tabla['area_grid']
tabla = tabla.sort_values(by=['id','peso'],ascending=False)
tabla = tabla.groupby('id').apply(lambda t: t.drop_duplicates(subset=['id'], keep='first'))

tabla.head()

,,WKT,id,area_grid,link,codpcia,departamen,provincia,color_psid,area_depto,area_union,peso
id,,,,,,,,,,,,
12841912.0,331500,MULTIPOLYGON (((-60.2079316412608 -34.06080769...,12841912.0,309579.798230,06714,06,Salto,Buenos Aires,1.0,1.604665e+09,309579.798230,1.0
12841913.0,331496,MULTIPOLYGON (((-60.2075642400061 -34.06618911...,12841913.0,309579.790700,06714,06,Salto,Buenos Aires,1.0,1.604665e+09,309579.790700,1.0
12841914.0,331497,MULTIPOLYGON (((-60.2071967412536 -34.07157053...,12841914.0,309579.783197,06714,06,Salto,Buenos Aires,1.0,1.604665e+09,309579.783197,1.0
12841915.0,331498,MULTIPOLYGON (((-60.2068291449782 -34.07695194...,12841915.0,309579.775702,06714,06,Salto,Buenos Aires,1.0,1.604665e+09,309579.775702,1.0
12841916.0,331499,MULTIPOLYGON (((-60.2064614511552 -34.08233335...,12841916.0,309579.768231,06714,06,Salto,Buenos Aires,1.0,1.604665e+09,309579.768231,1.0


In [5]:
tabla.to_csv('grilla_x_depto.csv',index=False)

In [9]:
tabla2 = tabla.reindex(columns=['id','link','codpcia','departamen','provincia']).reset_index(drop=True)

In [10]:
tabla2.shape

(110151, 5)

In [12]:
grid2 = grid.merge(tabla2,on='id',how='left')

In [15]:
tabla2.departamen.isnull().sum()

847

In [16]:
grid2.to_file('grilla_con_depto')